# Step1: Using PycisTopic to preprocess scATAC data

## Importing libraries

In [ ]:
import os
import sys
import subprocess
import pycisTopic
pycisTopic.__version__
import subprocess
from pycisTopic.cistopic_class import *
from pycisTopic.utils import *
from pycisTopic.lda_models import * 
import anndata as ad
import scanpy as sc

## Load Params

In [ ]:
# Determine the folder in which the code is executed
WORKING_DIR = os.getcwd()
sys.path.append(os.path.abspath( WORKING_DIR))

In [ ]:
%run -i ../../globalParams.py #GlobalParams
%run -i ../../sampleParams.py #sampleParams
%run -i ./analysisParams.py #AnalysisParams

## Set Up

In [ ]:
out_dir = PATH_ANALYSIS_OUTPUT
os.makedirs(out_dir, exist_ok = True)

In [ ]:
# Create the dictionnary of fragments from the fragment directory:
file_list = os.listdir(PATH_TO_FRAGMENTS_FILES)

# Filter out the .gz files but exclude .gz.tbi files
gz_files = [f for f in file_list if f.endswith('.gz') and not f.endswith('.gz.tbi')]

# Create the fragments_dict with keys based on the sample identifiers
fragments_dict = {f.split('_')[0].replace('-', ''): os.path.join(PATH_TO_FRAGMENTS_FILES, f) for f in gz_files}

# Print the resulting dictionary
fragments_dict


## Getting pseudobulk profiles from cell annotations

In [ ]:
#read the barcode to cell type annotation
import pandas as pd
cell_data = pd.read_csv(PATH_TO_CELLDATA_CSV,index_col=0)

#Add columns sample_id and barcodes
cell_data['sample_id'] = cell_data['sample']
cell_data['barcode'] = cell_data.index.str.split('_').str[1]


In [ ]:
cell_data

In [ ]:
#Rename fragments_dict to have the right names
# Create a mapping from the existing keys in fragments_dict to the sample_id in cell_data
sample_mapping = dict(cell_data[['sample', 'orig.ident']].drop_duplicates().values)
# Reverse the mapping to use sample as keys(since they correspond to the current keys in fragments_dict)
sample_mapping = {v.replace('-', ''): k for k, v in sample_mapping.items()}
# Create a new dictionary with the updated keys
fragments_dict = {sample_mapping.get(k, k): v for k, v in fragments_dict.items()}

# Print the new dictionary to verify
fragments_dict

### Download the chromosome size

In [ ]:
chromsizes = pd.read_table(
    "http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes",
    header = None,
    names = ["Chromosome", "End"]
)
chromsizes.insert(1, "Start", 0)
chromsizes.head()

## Generate Pseudobulk ATAC-seq profiles

In [ ]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk

os.makedirs(os.path.join(out_dir, "consensus_peak_calling"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"), exist_ok = True)


bw_paths, bed_paths = export_pseudobulk(
    input_data = cell_data,
    variable = CELL_TYPE_COLNAME,
    sample_id_col = SAMPLE_ID_COLNAME,
    chromsizes = chromsizes,
    bed_path = os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"),
    bigwig_path = os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"),
    path_to_fragments = fragments_dict,
    n_cpu = 22,
    normalize_bigwig = True,
    temp_dir = "/tmp",
    split_pattern = "_"
)

In [ ]:
#Save the paths to the disk
with open(os.path.join(out_dir, "consensus_peak_calling/bw_paths.tsv"), "wt") as f:
    for v in bw_paths:
        _ = f.write(f"{v}\t{bw_paths[v]}\n")
with open(os.path.join(out_dir, "consensus_peak_calling/bed_paths.tsv"), "wt") as f:
    for v in bed_paths:
        _ = f.write(f"{v}\t{bed_paths[v]}\n")

## Infering consensus peaks

### Peak calling

In [ ]:
bw_paths = {}
with open(os.path.join(out_dir, "consensus_peak_calling/bw_paths.tsv")) as f:
    for line in f:
        v, p = line.strip().split("\t")
        bw_paths.update({v: p})

bed_paths = {}
with open(os.path.join(out_dir, "consensus_peak_calling/bed_paths.tsv")) as f:
    for line in f:
        v, p = line.strip().split("\t")
        bed_paths.update({v: p})


from pycisTopic.pseudobulk_peak_calling import peak_calling
macs_path = "macs2"

os.makedirs(os.path.join(out_dir, "consensus_peak_calling/MACS"), exist_ok = True)

narrow_peak_dict = peak_calling(
    macs_path = macs_path,
    bed_paths = bed_paths,
    outdir = os.path.join(os.path.join(out_dir, "consensus_peak_calling/MACS")),
    genome_size = 'hs',
    n_cpu = 10,
    input_format = 'BEDPE',
    shift = 73,
    ext_size = 146,
    keep_dup = 'all',
    q_value = 0.05,
    #_temp_dir = '/scratch/leuven/330/vsc33053/ray_spill'
)

### Derive the consensus peaks

In [ ]:
from pycisTopic.iterative_peak_calling import get_consensus_peaks
# Other param
peak_half_width=250
path_to_blacklist=PATH_TO_BLACK_LIST
# Get consensus peaks
consensus_peaks = get_consensus_peaks(
    narrow_peaks_dict = narrow_peak_dict,
    peak_half_width = peak_half_width,
    chromsizes = chromsizes,
    path_to_blacklist = path_to_blacklist)

In [ ]:
consensus_peaks.to_bed(
    path = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed"),
    keep =True,
    compression = 'infer',
    chain = False)

## QC

### Download database

In [ ]:
#This need to adapt the code of docker file (see docker file)
!pycistopic tss gene_annotation_list | grep Human

!mkdir -p {out_dir}/qc

!pycistopic tss get_tss \
    --output {out_dir}/qc/tss.bed \
    --name "hsapiens_gene_ensembl" \
    --to-chrom-source ucsc \
    --ucsc hg38

#### Calculate QC metrics

In [ ]:
regions_bed_filename = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed")
tss_bed_filename = os.path.join(out_dir, "qc", "tss.bed")

pycistopic_qc_commands_filename = "pycistopic_qc_commands.txt"

# Create text file with all pycistopic qc command lines.
with open(pycistopic_qc_commands_filename, "w") as fh:
    for sample, fragment_filename in fragments_dict.items():
        print(
            "pycistopic qc",
            f"--fragments {fragment_filename}",
            f"--regions {regions_bed_filename}",
            f"--tss {tss_bed_filename}",
            f"--output {os.path.join(out_dir, 'qc')}/{sample}",
            sep=" ",
            file=fh,
        )

#Then run this in command line or go straight to the next cell for un paralleled process:
#cat pycistopic_qc_commands.txt | parallel -j 4 {}

In [ ]:
import subprocess

# Open and read the file line by line
with open(pycistopic_qc_commands_filename, "r") as file:
    for line in file:
        # Strip any leading/trailing whitespace
        command = line.strip()
        
        # Skip empty lines
        if not command:
            continue
        
        # Execute the command
        result = subprocess.run(command, shell=True)
        
        # Check for errors
        if result.returncode != 0:
            print(f"Command failed with return code {result.returncode}: {command}")


In [ ]:
from pycisTopic.plotting.qc_plot import plot_sample_stats, plot_barcode_stats
import matplotlib.pyplot as plt

In [ ]:
for sample_id in fragments_dict:
    fig = plot_sample_stats(
        sample_id = sample_id,
        pycistopic_qc_output_dir = os.path.join(out_dir, 'qc')
    )

In [ ]:
#Inspect the quality treshold automaticly identified
from pycisTopic.qc import get_barcodes_passing_qc_for_sample
sample_id_to_barcodes_passing_filters = {}
sample_id_to_thresholds = {}
for sample_id in fragments_dict:
    (
        sample_id_to_barcodes_passing_filters[sample_id],
        sample_id_to_thresholds[sample_id]
    ) = get_barcodes_passing_qc_for_sample(
            sample_id = sample_id,
            pycistopic_qc_output_dir = os.path.join(out_dir, 'qc'),
            unique_fragments_threshold = None, # use automatic thresholding
            tss_enrichment_threshold = None, # use automatic thresholding
            frip_threshold = 0,
            use_automatic_thresholds = True,
    )

In [ ]:
for sample_id in fragments_dict:
    fig = plot_barcode_stats(
        sample_id = sample_id,
        pycistopic_qc_output_dir = os.path.join(out_dir, 'qc'),
        bc_passing_filters = sample_id_to_barcodes_passing_filters[sample_id],
        detailed_title = False,
        **sample_id_to_thresholds[sample_id]
    )

### Creating a cis-topic object

In [ ]:
path_to_regions = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed")
path_to_blacklist = PATH_TO_BLACK_LIST
pycistopic_qc_output_dir = os.path.join(out_dir,"qc")
#os.makedirs(pycistopic_qc_output_dir, exist_ok = True)

from pycisTopic.cistopic_class import create_cistopic_object_from_fragments
import polars as pl

cistopic_obj_list = []
for sample_id in fragments_dict:
    sample_metrics = pl.read_parquet(
        os.path.join(pycistopic_qc_output_dir, f'{sample_id}.fragments_stats_per_cb.parquet')
    ).to_pandas().set_index("CB").loc[ sample_id_to_barcodes_passing_filters[sample_id] ]
    cistopic_obj = create_cistopic_object_from_fragments(
        path_to_fragments = fragments_dict[sample_id],
        path_to_regions = path_to_regions,
        path_to_blacklist = path_to_blacklist,
        metrics = sample_metrics,
        valid_bc = sample_id_to_barcodes_passing_filters[sample_id],
        n_cpu = 1,
        project = sample_id,
        split_pattern = '-'
    )
    cistopic_obj_list.append(cistopic_obj)


In [ ]:
#Merge into one single cisTopic
cistopic_obj = merge(cistopic_obj_list)

In [ ]:
cistopic_obj.cell_data

In [ ]:
cell_data.index

In [ ]:
import pickle
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

In [ ]:
# Make cell_data compatible
# Split the cell_data index to remove the sample prefix and change the suffix
new_index = cell_data.index.to_series().apply(
    lambda x: x.split('_')[1] + '-' + cell_data.loc[x, 'sample'] + '___' + cell_data.loc[x, 'sample']
)

# Update the index of cell_data
cell_data.index = new_index

# Verify the changes
print(cell_data.head())


In [ ]:
#Keep only the cells that are in both cell_data and cistopic object
# Ensure the new index is set for cell_data
new_index = cell_data.index

# Get the index from cistopic_obj
cistopic_index = cistopic_obj.cell_data.index

# Find the intersection (overlap) between the two indices
overlap = new_index.intersection(cistopic_index)

# Convert the overlapping indices to a list
overlap_list = overlap.to_list()

# Subset the cistopic_obj using the `subset` function to keep only cells in overlap
cistopic_obj = cistopic_obj.subset(cells=overlap_list, copy=True, split_pattern='___')

In [ ]:
cell_data = cell_data.loc[cell_data.index.isin(overlap)]

### Adding metadata to the cisTopic object

In [ ]:
cistopic_obj.add_cell_data(cell_data, split_pattern='_')
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

In [ ]:
cistopic_obj.cell_data

### Running scrublet (Optionnal)

In [ ]:
if DO_RUN_SCRUBLET:
    import scrublet as scr
    scrub = scr.Scrublet(cistopic_obj.fragment_matrix.T, expected_doublet_rate=0.1)
    doublet_scores, predicted_doublets = scrub.scrub_doublets()
    scrub.plot_histogram();
    scrub.call_doublets(threshold=0.22)
    scrub.plot_histogram();
    scrublet = pd.DataFrame([scrub.doublet_scores_obs_, scrub.predicted_doublets_], columns=cistopic_obj.cell_names, index=['Doublet_scores_fragments', 'Predicted_doublets_fragments']).T
    cistopic_obj.add_cell_data(scrublet, split_pattern = '-')
    sum(cistopic_obj.cell_data.Predicted_doublets_fragments == True)
    pickle.dump(cistopic_obj,open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb"))
    # Remove doublets
    singlets = cistopic_obj.cell_data[cistopic_obj.cell_data.Predicted_doublets_fragments == False].index.tolist()
    # Subset cisTopic object
    cistopic_obj_noDBL = cistopic_obj.subset(singlets, copy=True, split_pattern='-')
    print(cistopic_obj_noDBL)
    pickle.dump(cistopic_obj,open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb") )
    

## Run models

In [ ]:
#Parallel LDA with MALLET

import os
# Define the path to the tar file and the target directory
!wget https://github.com/mimno/Mallet/releases/download/v202108/Mallet-202108-bin.tar.gz

# Run the tar command to extract the contents to the specified directory
!tar -xf "Mallet-202108-bin.tar.gz" -C {PATH_ANALYSIS_OUTPUT}

In [ ]:
!mkdir -p {PATH_ANALYSIS_OUTPUT}/scratch/leuven/330/vsc33053/ray_spill/mallet/tutorial/

In [ ]:
os.chdir(out_dir)
os.environ['MALLET_MEMORY'] = '200G'
from pycisTopic.lda_models import run_cgs_models_mallet
# Configure paths 
mallet_path= os.path.join(out_dir, "Mallet-202108/bin/mallet")
TMP_PATH = os.path.join(out_dir, "scratch/leuven/330/vsc33053/ray_spill/mallet/tutorial")
SAVE_PATH = os.path.join(out_dir,"scratch/leuven/330/vsc33053/ray_spill/mallet/tutorial")

!mkdir -p /tmp/scratch/leuven/330/vsc33053/ray_spill/mallet/tutorial/


TMP_PATH = os.path.join("/tmp", "scratch/leuven/330/vsc33053/ray_spill/mallet/tutorial")
#SAVE_PATH = os.path.join("/tmp","scratch/leuven/330/vsc33053/ray_spill/mallet/tutorial")


# Run models
models=run_cgs_models_mallet(
    cistopic_obj,
    n_topics=[2, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
    n_cpu= 40,
    n_iter=500,
    random_state=555,
    alpha=50,
    alpha_by_topic=True,
    eta=0.1,
    eta_by_topic=False,
    tmp_path= TMP_PATH,
    save_path= SAVE_PATH,
    mallet_path= mallet_path,
)


In [ ]:
pickle.dump(
    models,
    open(os.path.join(out_dir, "models.pkl"), "wb")
)

## Model selection

In [ ]:
from pycisTopic.lda_models import evaluate_models
model = evaluate_models(
    models,
    select_model = 40,
    return_model = True
)

In [ ]:
cistopic_obj.add_LDA_model(model)

In [ ]:
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

# Clustering and visualization

In [ ]:
from pycisTopic.clust_vis import (
    find_clusters,
    run_umap,
    run_tsne,
    plot_metadata,
    plot_topic,
    cell_topic_heatmap
)

In [ ]:
find_clusters(
    cistopic_obj,
    target  = 'cell',
    k = 10,
    res = [0.6, 1.2, 3],
    prefix = 'pycisTopic_',
    scale = True,
    split_pattern = '-'
)

In [ ]:
#run_umap(
#    cistopic_obj,
#    target  = 'cell', scale=True)

In [ ]:
run_tsne(
    cistopic_obj,
    target  = 'cell', scale=True)

In [ ]:
# Plotting metadata
plot_metadata(
    cistopic_obj,
    reduction_name='tSNE',
    variables=['sample_id', 'pycisTopic_leiden_10_0.6', 'pycisTopic_leiden_10_1.2', 'pycisTopic_leiden_10_3'],
    target='cell', num_columns=4,
    text_size=10,
    dot_size=5)

In [ ]:
# Ploting continuous values
plot_metadata(
    cistopic_obj,
    reduction_name='tSNE',
    variables=['log10_unique_fragments_count', 'tss_enrichment', 'fraction_of_fragments_in_peaks'],
    target='cell', num_columns=4,
    text_size=10,
    dot_size=5)

In [ ]:
# cell-topic contribution
plot_topic(
    cistopic_obj,
    reduction_name = 'tSNE',
    target = 'cell',
    num_columns=5
)

In [ ]:
import matplotlib.colors as mcolors
# Example color dictionary (you can customize this)
color_dictionary = {
    'seurat_clusters': {cluster: mcolors.to_hex(mcolors.CSS4_COLORS[color]) for cluster, color in zip(sorted(cistopic_obj.cell_data['seurat_clusters'].dropna().unique()), mcolors.CSS4_COLORS)}
}


# Plot the heatmap with the selected cells
cell_topic_heatmap(
    cistopic_obj,
    variables=['seurat_clusters'],
    scale=False,
    legend_loc_x=1.0,
    legend_loc_y=-1.2,
    legend_dist_y=-1,
    figsize=(10, 10),
    #selected_cells= selected_cells,  # Pass the filtered cells
    color_dictionary=color_dictionary 
)

## Topic binarization and QC

In [ ]:
from pycisTopic.topic_binarization import binarize_topics

In [ ]:
region_bin_topics_top_3k = binarize_topics(
    cistopic_obj, method='ntop', ntop = 3_000,
    plot=True, num_columns=5
)

In [ ]:
region_bin_topics_otsu = binarize_topics(
    cistopic_obj, method='otsu',
    plot=True, num_columns=5
)

In [ ]:
region_bin_topics_otsu = binarize_topics(
    cistopic_obj, method='otsu',
    plot=True, num_columns=5
)

In [ ]:
binarized_cell_topic = binarize_topics(
    cistopic_obj,
    target='cell',
    method='li',
    plot=True,
    num_columns=5, nbins=100)

### Compute the topic quality control metrics

#### This step can't be run with R_Cystopic inputs (No model.coherence)

In [ ]:
from pycisTopic.topic_qc import compute_topic_metrics, plot_topic_qc, topic_annotation
import matplotlib.pyplot as plt
from pycisTopic.utils import fig2img

In [ ]:
topic_qc_metrics = compute_topic_metrics(cistopic_obj)

In [ ]:
fig_dict={}
fig_dict['CoherenceVSAssignments']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Log10_Assignments', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['AssignmentsVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Log10_Assignments', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSRegions_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Regions_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSMarginal_dist']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Marginal_topic_dist', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSGini_index']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Gini_index', var_color='Gini_index', plot=False, return_fig=True)

In [ ]:
# Plot topic stats in one figure
fig=plt.figure(figsize=(40, 43))
i = 1
for fig_ in fig_dict.keys():
    plt.subplot(2, 3, i)
    img = fig2img(fig_dict[fig_]) #To convert figures to png to plot together, see .utils.py. This converts the figure to png.
    plt.imshow(img)
    plt.axis('off')
    i += 1
plt.subplots_adjust(wspace=0, hspace=-0.70)
plt.show()

In [ ]:
#If needed:
#cistopic_obj.cell_data['seurat_clusters'] = cistopic_obj.cell_data['seurat_clusters'].fillna('Not_Assigned')
#cistopic_obj.cell_data['seurat_clusters'] = cistopic_obj.cell_data['seurat_clusters'].astype(str)
topic_annot = topic_annotation(
    cistopic_obj,
    annot_var=CELL_TYPE_COLNAME ,
    binarized_cell_topic=binarized_cell_topic,
    general_topic_thr = 0.2
)

## Differentially Accessible Regions (DARs)

In [ ]:
from pycisTopic.diff_features import (
    impute_accessibility,
    normalize_scores,
    find_highly_variable_features,
    find_diff_features
)
import numpy as np

In [ ]:
imputed_acc_obj = impute_accessibility(
    cistopic_obj,
    selected_cells=None,
    selected_regions=None,
    scale_factor=10**6
)

In [ ]:
normalized_imputed_acc_obj = normalize_scores(imputed_acc_obj, scale_factor=10**4)

In [ ]:
#Identifying highly variable DARs (optionnal but speed up the rest of the process)
variable_regions = find_highly_variable_features(
    normalized_imputed_acc_obj,
    min_disp = 0.05,
    min_mean = 0.0125,
    max_mean = 3,
    max_disp = np.inf,
    n_bins=20,
    n_top_features=None,
    plot=True
)

In [ ]:
TMP_PATH = "/tmp"
markers_dict= find_diff_features(
    cistopic_obj,
    imputed_acc_obj,
    variable= CELL_TYPE_COLNAME,
    var_features=variable_regions,
    contrasts=None,
    adjpval_thr=0.05,
    log2fc_thr=np.log2(1.5),
    n_cpu=5,
    _temp_dir=TMP_PATH,
    split_pattern = '-'
)

In [ ]:
from pycisTopic.clust_vis import plot_imputed_features

In [ ]:
#plot_imputed_features(
#    cistopic_obj,
#    reduction_name='tSNE',
#    imputed_data=imputed_acc_obj,
#    features=[markers_dict[x].index.tolist()[0] for x in ['9.0', '13.0', '7.0', '30.0']],
#    scale=False,
#    num_columns=4
#)

In [ ]:
print("Number of DARs found:")
print("---------------------")
for x in markers_dict:
    print(f"  {x}: {len(markers_dict[x])}")

## Save region sets

In [ ]:
os.makedirs(os.path.join(out_dir, "region_sets"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "Topics_otsu"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "Topics_top_3k"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "DARs_cell_type"), exist_ok = True)

In [ ]:
from pycisTopic.utils import region_names_to_coordinates

In [ ]:
for topic in region_bin_topics_otsu:
    region_names_to_coordinates(
        region_bin_topics_otsu[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "Topics_otsu", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
for topic in region_bin_topics_top_3k:
    region_names_to_coordinates(
        region_bin_topics_top_3k[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "Topics_top_3k", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
for cell_type in markers_dict:
    region_names_to_coordinates(
        markers_dict[cell_type].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "DARs_cell_type", f"{cell_type}.bed"),
        sep = "\t",
        header = False, index = False
    )

## Gene Activity

In [ ]:
import pyranges as pr
from pycisTopic.gene_activity import get_gene_activity

In [ ]:
chromsizes = pd.read_table(os.path.join(out_dir, "qc", "hg38.chrom_sizes_and_alias.tsv"))
chromsizes

In [ ]:
chromsizes.rename({"# ucsc": "Chromosome", "length": "End"}, axis = 1, inplace = True)
chromsizes["Start"] = 0
chromsizes = pr.PyRanges(chromsizes[["Chromosome", "Start", "End"]])

In [ ]:
chromsizes

In [ ]:
pr_annotation = pd.read_table(
        os.path.join(out_dir, "qc", "tss.bed")
    ).rename(
        {"Name": "Gene", "# Chromosome": "Chromosome"}, axis = 1)
pr_annotation["Transcription_Start_Site"] = pr_annotation["Start"]
pr_annotation = pr.PyRanges(pr_annotation)
pr_annotation

In [ ]:
gene_act, weigths = get_gene_activity(
    imputed_acc_obj,
    pr_annotation,
    chromsizes,
    use_gene_boundaries=True, # Whether to use the whole search space or stop when encountering another gene
    upstream=[1000, 100000], # Search space upstream. The minimum means that even if there is a gene right next to it
                             # these bp will be taken (1kbp here)
    downstream=[1000,100000], # Search space downstream
    distance_weight=True, # Whether to add a distance weight (an exponential function, the weight will decrease with distance)
    decay_rate=1, # Exponent for the distance exponential funciton (the higher the faster will be the decrease)
    extend_gene_body_upstream=10000, # Number of bp upstream immune to the distance weight (their value will be maximum for
                          #this weight)
    extend_gene_body_downstream=500, # Number of bp downstream immune to the distance weight
    gene_size_weight=False, # Whether to add a weights based on the length of the gene
    gene_size_scale_factor='median', # Dividend to calculate the gene size weigth. Default is the median value of all genes
                          #in the genome
    remove_promoters=False, # Whether to remove promoters when computing gene activity scores
    average_scores=True, # Whether to divide by the total number of region assigned to a gene when calculating the gene
                          #activity score
    scale_factor=1, # Value to multiply for the final gene activity matrix
    extend_tss=[10,10], # Space to consider a promoter
    gini_weight = True, # Whether to add a gini index weigth. The more unique the region is, the higher this weight will be
    return_weights= True, # Whether to return the final weights
    project='Gene_activity') # Project name for the gene activity object

In [ ]:
DAG_markers_dict= find_diff_features(
    cistopic_obj,
    gene_act,
    variable= CELL_TYPE_COLNAME,
    var_features=None,
    contrasts=None,
    adjpval_thr=0.05,
    log2fc_thr=np.log2(1.5),
    n_cpu=5,
    _temp_dir=TMP_PATH,
    split_pattern = '-')

In [ ]:
plot_imputed_features(
    cistopic_obj,
    reduction_name='tSNE',
    imputed_data=gene_act,
    features=['GATA3', 'TBX21', 'EOMES'], #NK
    scale=True,
    num_columns=4
)

In [ ]:
print("Number of DAGs found:")
print("---------------------")
for x in markers_dict:
    print(f"  {x}: {len(DAG_markers_dict[x])}")

## Label Transfer (useless)

In [ ]:
from pycisTopic.loom import export_region_accessibility_to_loom, export_gene_activity_to_loom

In [ ]:
# Pre-saving the before the loom saving in the PreSave_loom document
# Make sure the doc exist 
os.makedirs(os.path.join(out_dir, "PreSave_loom"), exist_ok=True)


## Pre-Saving for Loom

In [ ]:
# Save the imputed_acc_obj using pickle
pickle.dump(
    imputed_acc_obj,
    open(os.path.join(out_dir, "PreSave_loom", "imputed_acc_obj.pkl"), "wb")
)

# Save the cistopic_obj object using pickle
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "PreSave_loom", "cistopic_obj.pkl"), "wb")
)


# Save the region_bin_topics_otsu using pickle
pickle.dump(
    region_bin_topics_otsu,
    open(os.path.join(out_dir, "PreSave_loom", "region_bin_topics_otsu.pkl"), "wb")
)

# Save the binarized_cell_topic using pickle
pickle.dump(
    binarized_cell_topic,
    open(os.path.join(out_dir, "PreSave_loom", "binarized_cell_topic.pkl"), "wb")
)

# Save the cluster_markers using pickle
pickle.dump(
    cluster_markers,
    open(os.path.join(out_dir, "PreSave_loom", "cluster_markers.pkl"), "wb")
)

# Save the gene_act using pickle
pickle.dump(
    gene_act,
    open(os.path.join(out_dir, "PreSave_loom", "gene_act.pkl"), "wb")
)

## Exporting to Loom

In [ ]:
# Here it may be recommanded to restart kernel and load the saved data before proceeding to the Loom saving

In [ ]:
os.makedirs(os.path.join(out_dir, "loom"), exist_ok=True)

In [ ]:
export_region_accessibility_to_loom(
    accessibility_matrix = imputed_acc_obj,
    cistopic_obj = cistopic_obj,
    binarized_topic_region = region_bin_topics_otsu,
    binarized_cell_topic = binarized_cell_topic,
    selected_cells = cistopic_obj.projections['cell']['tSNE'].index.tolist(),
    out_fname = os.path.join(out_dir, "loom", "NK_Tumor_MultiSample_pycisTopic_region_accessibility.loom"),
    cluster_annotation = [CELL_TYPE_COLNAME],
    cluster_markers = cluster_markers,
    tree_structure = ('NK_Tumor_MultiSample', 'pycisTopic', 'noDBL_all'),
    title = 'Tutorial - Region accessibility all',
    nomenclature = "hg38",
    split_pattern = '-'
)

In [ ]:
export_gene_activity_to_loom(
    gene_activity_matrix = gene_act,
    cistopic_obj = cistopic_obj,
    out_fname = os.path.join(out_dir, "loom", "NK_Tumor_pycisTopic_gene_activity.loom"),
    cluster_annotation = [CELL_TYPE_COLNAME],
    cluster_markers = cluster_markers,
    tree_structure = ('NK_Tumor_MultiSample', 'pycisTopic', 'ATAC'),
    title = 'NK_Tumor - Gene activity',
    nomenclature = "hg38",
    split_pattern = '-'
)

## Stop the notebook

In [ ]:
if STOP_THE_NOTEBOOK_HERE:
  raise Exception("Analysis stopped here"

In [ ]:
#Correct the cell_data slot to match the cell
#Be carefull to the names of the cells in cell_data vs incitopic_obj
import pandas as pd
cell_data = pd.read_csv(PATH_TO_CELLDATA_CSV, index_col = 0)
# print(cell_data)

# Split the cell_data index to remove the sample prefix and change the suffix
new_index = cell_data.index.to_series().apply(
    lambda x: x.split('_')[1] + '-' + cell_data.loc[x, 'sample'] + '___' + cell_data.loc[x, 'sample']
)

# Update the index of cell_data
cell_data.index = new_index

# Verify the changes
print(cell_data.head())


## Load the data to go straight to Loom saving

In [ ]:
# Load the data
import os
import sys
import subprocess
import pycisTopic
pycisTopic.__version__
import subprocess
from pycisTopic.cistopic_class import *
from pycisTopic.utils import *
from pycisTopic.lda_models import * 
import anndata as ad
import scanpy as sc
import pandas as pd
import pickle

# Determine the folder in which the code is executed
WORKING_DIR = os.getcwd()
sys.path.append(os.path.abspath( WORKING_DIR))

#Run the basic
%run -i ../../globalParams.py #GlobalParams
%run -i ../../sampleParams.py #sampleParams
%run -i ./analysisParams.py #AnalysisParams

#Define outdir
out_dir = PATH_ANALYSIS_OUTPUT
os.makedirs(out_dir, exist_ok = True)

# Load the imputed_acc_obj
with open(os.path.join(out_dir, "PreSave_loom", "imputed_acc_obj.pkl"), "rb") as file:
    imputed_acc_obj = pickle.load(file)

# Load the cistopic_obj
with open(os.path.join(out_dir, "PreSave_loom", "cistopic_obj.pkl"), "rb") as file:
    cistopic_obj = pickle.load(file)

# Load the region_bin_topics_otsu
with open(os.path.join(out_dir, "PreSave_loom", "region_bin_topics_otsu.pkl"), "rb") as file:
    region_bin_topics_otsu = pickle.load(file)

# Load the binarized_cell_topic
with open(os.path.join(out_dir, "PreSave_loom", "binarized_cell_topic.pkl"), "rb") as file:
    binarized_cell_topic = pickle.load(file)

# Load the cluster_markers
with open(os.path.join(out_dir, "PreSave_loom", "cluster_markers.pkl"), "rb") as file:
    cluster_markers = pickle.load(file)


# Load the gene_act
with open(os.path.join(out_dir, "PreSave_loom", "gene_act.pkl"), "rb") as file:
    gene_act = pickle.load(file)


## Correct the cell data

In [ ]:
cistopic_obj.cell_data

In [ ]:
markers_dict